# 命名实体抽取

### 概要：
- `include("../src/prefixtree.jl")` 导入字典树，`get_subject` 抽取命名实体
- 训练集准确率 `99.4%`，验证集可完全提取

细节：
- 训练集识别结果
    - 13992 个准确识别
    - 65 个无法识别错误（以中文为主）
    - 20 个无法识别，有以下 7 种，除了前 2 个外，均为乱码问题
    
     ```jl
     "who married the person of T-related eventhose Were the Days (song)?"
     "what is children of the person of T related eventhose Were the Days (song)?"
     "Savez-vous qu'avant produits liés à l'ENTITÉest?"
     "Savez-vous quels sont les pays participants"
     "Saviez-vous que la série du produits liés à l'ENTITÉest?"
     "Savez-vous ce qu’est la dynastie ?"
     "produits liés à l'ENTITÉbasé sur ce"
     ```
- 验证集识别情况：
    - 有 3 个无法识别，且均为问题错位导致，已在 `extract` 阶段处理
    - 导入方式
    ```
    valid_ques_ner = Vector{String}(split(
        strip(read(open("extract/valid_data_ner.txt", "r"), String)), '\n'))
    ```

### 导入

In [8]:
include("../src/extractdata.jl") # 导入 extract 中的数据
include("../src/prefixtree.jl") # 前缀树工具

get_subject (generic function with 1 method)

In [9]:
# 检查准确性
valids, fails, nomatch = String[], String[], String[]
for (que, ner) in train_ques_ner
    sub = get_subject(que)
    isempty(sub) && (push!(nomatch, que); continue)
    sub == ner ? push!(valids, que) : push!(fails, que)
end
length.([valids, fails, nomatch])

3-element Vector{Int64}:
 13992
    65
    20

In [10]:
# 准确率
13992 / length(train_ques_ner) * 100

99.39617816296085

In [11]:
# 验证集有效提取的数目
filter(i->isempty(get_subject(i)), valid_ques)

String[]

In [13]:
# 提取验证集
open("extract/valid_data_ner.txt", "w") do io
    for que in valid_ques
        println(io, que, '\t', get_subject(que))
    end
end